In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
def get_links_df(url):
    soup = BeautifulSoup(requests.get(url).text,"html.parser")
    
    table = soup.find_all("table", class_ = "sortable")[0]
    table_rows = table.find_all("tr")[1:]
    anchors = [_.find_all("a") for _ in table_rows if len(_.find_all("a"))>0]
    
    cols = ["link", "name"]
    links_df = pd.DataFrame([(anchor[0].get("href"), anchor[0].text) for anchor in anchors], columns=cols)
    
    return links_df

In [4]:
def cleanResults(result):
    event_name = result["event"]
    df = result["df"]
    df.columns = [_[1] for _ in df.columns]

    split_by_card = [(i, df.iloc[i,0]) for i in df[df.eq(df.iloc[:, 0], axis=0).all(axis=1)].index]
    if 0 not in [_[0] for _ in split_by_card]:
        split_by_card = [(0, "Main card"), *split_by_card]

    for i in range(len(split_by_card)-1):
        split_by_card[i] = (split_by_card[i][0], split_by_card[i+1][0], split_by_card[i][1])

    split_by_card[-1] = (split_by_card[-1][0], len(df), split_by_card[-1][1])

    sdfs = []
    for start, end, card in split_by_card:
        sdf = df.iloc[start:end, :]
        sdf = sdf.assign(fight_card = card)
        sdfs.append(sdf)
    
    df = pd.concat(sdfs).drop([i[0] for i in split_by_card[1:]]).reset_index(drop=True)

    cols2rename = {
        x: x.lower().replace(' ', '_') for x in df.columns
    }

    cols2rename = {
        **cols2rename,
        **{
            "Unnamed: 1_level_1": "winner",
            "Unnamed: 3_level_1": "loser",
        },
    }

    df.rename(columns=cols2rename, inplace=True)

    df.drop(columns=["unnamed:_2_level_1"], axis=1, inplace=True)

    df = df.assign(event_name = event_name)

    return df

In [21]:
def event_details(event, table_class, url):
    event_name = event.text.replace("[edit]", "")
    tables = event.find_next("table", class_ = table_class)
    if tables != None:
        df = pd.read_html(str(tables))
    else:
        df = pd.read_html(str(event.find_next("table", class_ = "wikitable")))
    
    df = df[0]
    df = cleanResults(result={"event": event_name, "df": df})
    df = df.assign(link = url)

    return df

In [22]:
def getData_bellator(url):
    dfs = []
    soup = BeautifulSoup(requests.get(url).text, features="lxml")
    header_patterns = ["bellator"]
    pattern_match = lambda x: len([_ for _ in header_patterns if _ in x.lower()]) > 0

    # event_headers = soup.find_all(lambda tag: tag.name == "h2" and "one" in tag.text.lower() and "cancelled" not in tag.text.lower())
    event_headers = [
        header for header in soup.find_all(
        lambda tag: 
        tag.name == "h2" and "cancelled" not in tag.text.lower() and "tournament" not in tag.text.lower()
        ) if pattern_match(header.text)
        ]

    if not event_headers:
        event_headers = soup.find_all(lambda tag: tag.name == "h1" and "bellator" in tag.text.lower() and "cancelled" not in tag.text.lower())
    
    table_classes = [_.get("class") for _ in event_headers[0].find_all_next("table")]

    table_class = "wikitable" if "toccolours" not in [x for xs in list(filter(lambda x: x!=None, table_classes)) for x in xs] else "toccolours"

    for event in event_headers:
        try:
            df = event_details(event, table_class, url)
            dfs.append(df)
        except:
            print(event.text)

    return pd.concat(dfs, ignore_index=True)

In [6]:
url = 'https://en.wikipedia.org/wiki/List_of_Bellator_MMA_events'
links_df = get_links_df(url)

clean_link = lambda x: f"https://en.wikipedia.org/{x.split('#')[0]}"
links_df["link_clean"] = links_df["link"].apply(clean_link)
links = links_df["link_clean"].value_counts().reset_index().iloc[:,0]
links = links.to_list()

dfs = []
failed_links = []

In [11]:
[link for link in links if "bellator_22" in link.lower()]

['https://en.wikipedia.org//wiki/Bellator_22']

In [24]:
df = getData_bellator('https://en.wikipedia.org//wiki/Bellator_22')

Bellator 26[edit]
Bellator 27[edit]
Bellator 28[edit]
Bellator 29[edit]
Bellator 30[edit]
Bellator 31[edit]
Bellator 32[edit]
Bellator 33[edit]
Bellator 34[edit]


In [26]:
pd.concat([None,None], ignore_index=True)

ValueError: All objects passed were None